# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [19]:
#Importing reqiured libraries
import selenium
import pandas as pd
from selenium import webdriver
import requests

In [20]:
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime
from bs4 import BeautifulSoup

In [44]:
#Connecting to web driver
driver = webdriver.Chrome(r"C://chromedriver.exe")
driver.maximize_window()

In [45]:
driver.get('http://amazon.in/')
driver.implicitly_wait(5)

In [46]:
# Let's enter the details in the search column
p_search=input("Enter product you want to search: ")
sear= driver.find_element_by_id("twotabsearchtextbox")
sear.send_keys(p_search)
sea_but=driver.find_element_by_id("nav-search-submit-button")
sea_but.click()

Enter product you want to search: laptops


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [34]:
# Let's create empty list to store data
bname=[]
pname=[]
rat=[]
nrat=[]
price=[]
ex_del=[]
ret_ex=[]
other=[]
avail=[]
p_url=[]

In [47]:
#Functions for getting products in all three pages
urls=[]
for k in range(3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        urls.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break

#Taking every URL one after one
for i in urls:
    driver.get(i)
    p_url.append(i)
    driver.implicitly_wait(5)
    
    
#Scraping the brand name
    try: bname.append(driver.find_element_by_id("productTitle").text.split(' ')[0]) 
    except: bname.append('-')
    driver.implicitly_wait(5)
    
#Scraping the name of product
    try: 
        string=''
        for i in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+i
        pname.append(string)
    except: pname.append('-')
    driver.implicitly_wait(5)
    
#Scraping the rating of the product
    try: rat.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except: rat.append('-')
    driver.implicitly_wait(5)
    
#Scraping the number of ratings
    try: nrat.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except: nrat.append('-')
    driver.implicitly_wait(5)
    
#Scraping the price of the product
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
    driver.implicitly_wait(5)
    
#Scraping the details of return/exchnage of the product
    try: 
        return_row=driver.find_element_by_xpath("//div[@class='a-row icon-farm-wrapper']").find_elements_by_xpath(".//div")
        found=False
        for i in return_row:
            if(i.get_attribute('data-name')=='RETURNS_POLICY'):
                found=True
                ret_ex.append(i.find_element_by_xpath(".//span[1]/div[2]/a").text)
        if(found==False):
            ret_ex.append('-')
    except: ret_ex.append('-')
    driver.implicitly_wait(5)
    
#Scraping the expected delivery date
    try: ex_del.append(driver.find_element_by_id("ddmDeliveryMessage").find_element_by_xpath(".//b").text)
    except: ex_del.append('-')
    driver.implicitly_wait(5)
    
#Scraping the availablity of the product
    try: 
        try: avail.append(driver.find_element_by_id("availability").find_element_by_xpath(".//span").text)
        except: avail.append(driver.find_element_by_id("deal_availability").find_element_by_xpath(".//div/span").text)
    except: avail.append('-')
    driver.implicitly_wait(5)
    
#Scraping other details
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")] 
        other.append(details[0])
    except: other.append('-')
    driver.implicitly_wait(5)        

In [50]:
# Let's create a DataFrame for our data
amazon_lap=pd.DataFrame()

In [53]:
amazon_lap['Brand Name']=bname
amazon_lap['Name of Product']=pname
amazon_lap['Rating']=rat
amazon_lap['No. of Ratings']=nrat
amazon_lap['Price']=price
amazon_lap['Return/Exchange']=ret_ex
amazon_lap['Expected Delivery']=ex_del
amazon_lap['Availability']=avail
amazon_lap['Other details']=other
amazon_lap['Product URL']=p_url

In [54]:
amazon_lap

,Brand Name,Name of Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,(Renewed),Dell Vostro 5481 14-inch Laptop (Core i5/4GB/...,-,-,"₹ 44,990.00",7 Days Replacement,"Tuesday, May 4",Only 1 left in stock.,Brand Dell---Manufacturer Dell Computers---Ser...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Acer,"Swift 5 14"" Full HD IPS Display with Touchscr...",4.4 out of 5 stars,13 ratings,"₹ 91,999.00",7 Days Replacement,"Tuesday, May 4",In stock.,Brand Acer---Manufacturer Nanchang HuaQin Elec...,https://www.amazon.in/gp/slredirect/picassoRed...
2,HP,15 Entry Level 15.6-inch HD Laptop (AMD 3020e...,3.8 out of 5 stars,470 ratings,"₹ 23,990.00",7 Days Replacement,April 30 - May 4,Only 2 left in stock.,"Brand HP---Manufacturer hp, HP---Model Name 15...",https://www.amazon.in/HP-15-6-inch-Laptop-Wind...
3,Dell,"Inspiron 3502 15.6"" HD Display Laptop (Pentiu...",4.0 out of 5 stars,6 ratings,"₹ 27,390.00",7 Days Replacement,May 3 - 4,"In stock on May 1, 2021.","Brand Dell---Manufacturer Dell India Pvt Ltd, ...",https://www.amazon.in/Dell-Inspiron-Integrated...
4,AVITA,Essential NE14A2INC433-CR 14-inch Laptop (Cel...,3.9 out of 5 stars,485 ratings,"₹ 17,990.00",7 Days Replacement,"Tuesday, May 4",In stock.,Brand AVITA---Manufacturer Alco Electronics Li...,https://www.amazon.in/AVITA-Essential-NE14A2IN...
...,...,...,...,...,...,...,...,...,...,...
155,ASUS,VivoBook 14 (2020) Intel Core i3-1005G1 10th ...,4.1 out of 5 stars,65 ratings,"₹ 39,990.00",7 Days Replacement,May 2 - 6,"In stock on April 30, 2021.","Brand ASUS---Manufacturer Asus, TECH FRONT ( C...",https://www.amazon.in/ASUS-VivoBook-i3-1005G1-...
156,Lenovo,V15 Intel Core i5 10th Gen 15-inch Full HD Th...,3.8 out of 5 stars,77 ratings,"₹ 43,490.00",7 Days Replacement,"Tuesday, May 4",In stock.,"Brand Lenovo---Manufacturer Lenovo, Wujiang Hi...",https://www.amazon.in/Lenovo-15-inch-Laptop-Wi...
157,Lenovo,E41-45 (82BF000JIH) Laptop (AMD A6-7350B/ 4GB...,3.5 out of 5 stars,35 ratings,"₹ 24,490.00",7 Days Replacement,"Tuesday, May 4",In stock.,"Brand Lenovo---Manufacturer Lenovo, Lenovo, Ch...",https://www.amazon.in/Lenovo-E41-45-82BF000JIH...
158,HP,15 11th Gen Intel Core i3 Thin & Light 15.6-i...,3.1 out of 5 stars,14 ratings,"₹ 36,990.00",7 Days Replacement,"Tuesday, May 4",In stock.,"Brand HP---Manufacturer hp, HP India Sales Pvt...",https://www.amazon.in/gp/slredirect/picassoRed...


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [67]:
#Connecting to web driver
driver = webdriver.Chrome(r"C://chromedriver.exe")
driver.maximize_window()

In [68]:
url=('https://www.google.com/imghp?hl=en')
driver.get(url)
driver.implicitly_wait(5)

In [69]:
#Function taking image name as an argument
def get_urls(keywords):
        driver.get(url)
        
#Details in the search column and click search button
        search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
        search_bar.send_keys(keywords)
        search_but=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
        search_but.click()
        driver.implicitly_wait(5)
        
        
#loading pages to scrape images
        for k in range(12):
            driver.find_element_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']").send_keys(Keys.PAGE_DOWN)
        time.sleep(5)
        picture=[image for image in driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']")[:100]]
        image_urls=[]
        print(len(picture))
        if(len(picture)==100):
            for i in picture:
                try:
                    i.click()
                    image_urls.append(i.get_attribute('href'))
                except:
                    image_urls.append('Not Available')
            return image_urls
        else:
            print('Image Not Available')

In [70]:
#Let's scrape the url's of all images
fru=get_urls('Fruits')
car=get_urls('Cars')
m_l=get_urls('Machine Learning')

100
100
100


In [71]:
#Creating a dataframe for storing data
img_url=pd.DataFrame({})
img_url['Fruits']=fru
img_url['Cars']=car
img_url['Machine Learning']=m_l

In [72]:
img_url

,Fruits,Cars,Machine Learning
0,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
1,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
2,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
3,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
4,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
...,...,...,...
95,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
96,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
97,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
98,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...


# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”.

In [73]:
#Connecting to web driver
driver = webdriver.Chrome(r"C://chromedriver.exe")

In [74]:
url=('https://www.flipkart.com/')
driver.get(url)
driver.implicitly_wait(5)

In [76]:
#Searching for smartphone on Flipkart
prod=input('Enter the product you want to buy : ')
driver.find_element_by_xpath('//input[@title="Search for products, brands and more"]').send_keys(str(prod))
driver.find_element_by_xpath('//button[@type="submit"]').click()
driver.implicitly_wait(3)

Enter the product you want to buy : smartphones


In [77]:
#Let's create an empty list to store the scrape data
product_urls=[]
bname=[]
pname=[]
pcolor=[]
ram=[]
rom=[]
p_camera=[]
s_camera=[]
dis_size=[]
dis_res=[]
proc=[]
proc_core=[]
batt=[]
price=[]

In [78]:
#Functions for getting products links
link=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')

#Let's scrape product URL
for i in link:
    product_urls.append(i.get_attribute('href'))
for i in product_urls:
    driver.get(i)
    driver.implicitly_wait(4)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    try:
        
#Let's scrape brand name
        title=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        bname.append(title[0])
    except:
        bname.append('-')
    try:
        
#Let's scrape smartohone name
        pname.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]').text)
    except:
        pname.append('-')
    try:
        
#Let's scrape smartphone colour
        pcolor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]').text)
    except:
        pcolor.append('-')
    try:
        
#Let's scrape smartphone RAM
        ram.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except:
        ram.append('-')
    try:
        
#Let's scrape smartphone ROM
        rom.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except:
        rom.append('-')
    try:
        
#Let's scrape smartphone primary camera details
        p_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except:
        p_camera.append('-')
    try:
        
#Let's scrape smartphone secondary camera details
        s_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except:
        s_camera.append('-')
    try:
        
#Let's scrape smartphone display size
        dis_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except:
        dis_size.append('-')
    try:
        
#Let's scrape smartphone screen resolution
        dis_res.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except:
        dis_res.append('-')
    try:
        
#Let's scrape smartphone processor
        proc.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except:
        proc.append('-')
    try:
        
#Let's scrape smartphone processor core
        proc_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except:
        proc_core.append('-')
    try:
        
#Let's scrape smartphone battery capacity
        batt.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except:
        batt.append('-')
    try:
        
#Let's scrape smartphone price
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except:
        price.append('-')

In [79]:
#Creating a dataframe for storing the data
flip=pd.DataFrame({})
flip['Brand Name']=bname
flip['Smartphone Name']=pname
flip['Smartphone Colour']=pcolor
flip['RAM']=ram
flip['Storage (ROM)']=rom
flip['Primray Camera']=p_camera
flip['Secondary Camera']=s_camera
flip['Display Size']=dis_size
flip['Display Resolution']=dis_res
flip['Processor']=proc
flip['Processor Cores']=proc_core
flip['Battery Capacity']=batt
flip['Price']=price
flip['Product URL']=product_urls

In [80]:
flip

,Brand Name,Smartphone Name,Smartphone Colour,RAM,Storage (ROM),Primray Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,Product URL
0,Infinix,Hot 10 Play,Aegean Blue,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/infinix-hot-10-play-a...
1,Infinix,Hot 10 Play,Morandi Green,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/infinix-hot-10-play-m...
2,Infinix,Hot 10 Play,Obsidian Black,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/infinix-hot-10-play-o...
3,REDMI,9i,Sea Blue,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,Brand Warranty of 1 Year Available for Mobile ...,"₹7,999",https://www.flipkart.com/redmi-9i-sea-blue-64-...
4,realme,C12,Power Silver,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.56 cm (6.52 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,6000 mAh,"₹7,999",https://www.flipkart.com/realme-c12-power-silv...
5,realme,C12,Power Blue,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.56 cm (6.52 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,6000 mAh,"₹7,999",https://www.flipkart.com/realme-c12-power-blue...
6,realme,C20,Cool Grey,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,76.4 mm,"₹6,799",https://www.flipkart.com/realme-c20-cool-grey-...
7,realme,C20,Cool Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,76.4 mm,"₹6,799",https://www.flipkart.com/realme-c20-cool-blue-...
8,realme,C21,Cross Blue,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,76.4 mm,"₹7,999",https://www.flipkart.com/realme-c21-cross-blue...
9,REDMI,9i,Nature Green,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,Brand Warranty of 1 Year Available for Mobile ...,"₹7,999",https://www.flipkart.com/redmi-9i-nature-green...


In [81]:
#Saving the dataframe into a csv file
flip.to_csv('flipkart-phones.csv')

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [82]:
#Connecting to web driver
driver = webdriver.Chrome(r"C://chromedriver.exe")

In [83]:
url=('https://www.google.com/maps/')
driver.get(url)
driver.implicitly_wait(5)

In [88]:
#Function to enter the city name that you want to search 
city_name = input('Enter the name of the City you want to search : ') 
city_search=driver.find_element_by_id('searchboxinput')
city_search.send_keys(city_name)
#Function for searching the entered city
driver.find_element_by_id('searchboxinput').send_keys(Keys.ENTER)
driver.implicitly_wait(6)

#Let's Scrape the entered city latitude and Longitude from google map
lati=driver.current_url.split('/@')[1].split(',')[0]
longi=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude of ',city_name,'is: ',lati)
print('Longitude of ',city_name,'is: ',longi)

Enter the name of the City you want to search : shimla
Latitude of  shimla is:  28.6359552
Longitude of  shimla is:  77.2308992


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [93]:
#Connecting to web driver
driver = webdriver.Chrome(r"C://chromedriver.exe")

In [94]:
url=('https://trak.in/')
driver.get(url)
driver.implicitly_wait(5)

In [95]:
# Entering the details in the search column
s_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(s_button)

In [96]:
#Creating empty lists for storing the data
Date=[]
Startup_Name=[]
Industry=[]
Location=[]
Investor=[]
Investment=[]
Amount=[]

In [97]:
#Creating a function for getting the details
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()
    
#Scraping the date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        Date.append(d.text)
        
#Scraping the details
    startup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in startup:
        Startup_Name.append(n.text)
    
#Scraping the industry details
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in industry:
        Industry.append(n.text)

#Scraping the location details
    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in location:
        Location.append(l.text)
    
#Scraping the investors detils
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in investor:
        Investor.append(n.text)
    
#Scraping the investment details
    investment = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in investment:
        Investment.append(n.text)
    
#Scraping the amount
    amount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amount:
        Amount.append(a.text)

In [99]:
# Let's create a DataFrame for our data
trak=pd.DataFrame({})
trak['Date']=Date
trak['Startup']=Startup_Name 
trak['Industry']=Industry
trak['Location']=Location
trak['Investor']=Investor
trak['Investment']=Investment
trak['Amount']=Amount

In [100]:
trak

,Date,Startup,Industry,Location,Investor,Investment,Amount
0,15/07/2020,Flipkart,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


# 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [108]:
#Connecting to web driver
driver = webdriver.Chrome(r"C://chromedriver.exe")

In [109]:
url=('https://www.digit.in/')
driver.get(url)
driver.implicitly_wait(5)

In [110]:
#Choosing top 10 from the menu
driver.find_element_by_xpath('//div[@class="menu"]/ul/li[3]/a').click()

In [115]:
#Chhosing top laptop in India
driver.find_element_by_xpath('//div[@class="Listbrand"]/ul/li[10]/a').click()

In [118]:
#Creating empty lists for storing the data
name = []
OS = []
display = []
processor = []
HDD = []
price = []
RAM = []
dimension = []
GPU = []
weight = []

In [119]:
#Scraping the Company Name
Name=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in Name:
    name.append(i.text)
    
#Scraping the OS type
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
#Scraping the display size
Display=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in Display:
    display.append(i.text)
    
#Scraping the processor type
Processor=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in Processor:
    processor.append(i.text)
processor

#Scraping the Memory
Memory=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")
Memory_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]") 
for i in range(len(Memory)):
        if Memory[i].text=='Memory':
            HDD.append(Memory_spec[i].text.split('/')[0])
            RAM.append(Memory_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')
            RAM.append('No details available')

#Scraping the weith
Weight=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
Weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(Weight)):
        if Weight[i].text=='Weight':
            weight.append(Weight_spec[i].text)
        
#Scraping the dimensions
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#Scraping the Graphic Processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)

#Scraping the other details 
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))

#Scraping a for loop to scrape the details from each page
for i in full_specs:
    driver.get(i)
    try:
        Price=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(Price.text)
    except NoSuchElementException:
        price.append("No details available")

In [122]:
# Let's create a DataFrame for our data
digit_top=pd.DataFrame({})
digit_top["Company Name: "]=Name
digit_top["OS: "]=OS
digit_top["Screen Display Size: "]=display
digit_top["HDD: "]=HDD
digit_top["RAM: "]=RAM
digit_top["Processor: "]=processor
digit_top["Weight: "]=weight
digit_top["Dimension (mm): "]=dimension
digit_top["Graphical Processor Unit: "]=GPU
digit_top["Price in Rs.: "]=price

In [123]:
digit_top

,Company Name:,OS:,Screen Display Size:,HDD:,RAM:,Processor:,Weight:,Dimension (mm):,Graphical Processor Unit:,Price in Rs.:
0,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 PRO,"17.3"" (UHD 3840X2160)",1 TB HDD,64 GBGB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,"379,990"
1,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 PRO,"17.3"" (FHD (1920 X 1080))",1 TB PCIe SSD,32GB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080,"422,000"
2,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,16 GBGB DDR4,INTEL I7-10750H 10TH GEN | 1.6GHZ,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti,"117,790"
3,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"14"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"164,990"
4,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1 TB SSD,8GB DDR4,9TH GENERATION CORE INTEL I7-9750H | 2.6 GHZ,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060,"79,990"
5,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,16GB DDR4,AMD RYZEN 7 QUAD CORE 3750H | 2.3 GHZ,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti,"79,990"
6,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512GB SSD,16 GBGB DDR4,9TH GEN INTEL CORE I7-8750H | 2.2 GHZ,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q),"239,990"
7,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME 64 BIT,"18.4"" (1920 X 1080)",1.5 TB SATA,64GB DDR4,INTEL CM238 CORE I7-7820HK+CM238 7TH GEN | 3.5GHZ,5.5,458 x 339 x 69,Dual GTX1070,"349,990"
8,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10,"15.6"" (3840 X 1100)",512 GB SSD,4 GBGB DDR4,INTEL CORE I7 10TH GEN 10875H | NA,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"244,990"
9,<selenium.webdriver.remote.webelement.WebEleme...,WINDOWS 10 HOME PLUS,15.6 MP | NA,1TB HDD,8GB DDR4,8TH GEN INTEL CORE I5-8300H | 2.3GHZ,2.53,22.7 X 380 X 258,NVidia GeForce GTX 1050,"61,990"
